# DB연결하기

In [3]:
import configparser
global HOST,PORT,DB_ID,DB_PW,MODEL_DIR,MODEL_NAME
conf_dir = '/home/cdsadmin/python_src/EY/Emart/conf/config.ini'
cfg = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
cfg.read(conf_dir)

HOST      = cfg['dbconnect']['host']
PORT      = int(cfg['dbconnect']['port'])
DB_ID     = cfg['dbconnect']['ID']
DB_PW     = cfg['dbconnect']['PW']
MODEL_DIR = cfg['event']['MODEL_DIR']

###==========================================================================###
import numpy as np
import pandas as pd 
import time,os,psutil
from hdbcli import dbapi ## hana DB client 
from IPython.core.display import display, HTML
import multiprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from collections import Counter
import seaborn as sns
from sklearn.ensemble   import RandomForestClassifier,GradientBoostingClassifier
from xgboost            import XGBClassifier
from lightgbm           import LGBMClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.linear_model    import LogisticRegression
from sklearn.metrics         import confusion_matrix,precision_score,recall_score,\
                                      f1_score,accuracy_score,roc_auc_score,roc_curve,auc,\
                                      classification_report,precision_recall_curve

def DB_Connection() :
    conn=dbapi.connect(HOST,PORT,DB_ID,DB_PW)
    return conn

def select_query(sql) :
    conn = DB_Connection()
    cnt = pd.read_sql(sql, conn)
    conn.close()
    
def load_model(filename):
    model_dir  = MODEL_DIR+filename
    load_model = joblib.load(model_dir)
    return load_model

def query_SQL( query ): 
    ## DB Connection
    conn=DB_Connection() ## DB 연결 정보
    ## Get a DataFrame 
    start_time = time.time()
    query_result = pd.read_sql(query, conn)
    ## Close Connection
    print( '---- %s seconds ------'%(time.time()-start_time))
    conn.close()
    
    return query_result

# 예측하고자하는 년월을 입력하면 앞전 3개월치의 데이터를 자동으로 로드해옴
#----------------------------------------------------------
# Configuration variable 
#----------------------------------------------------------
DNA_YM = '202009'
BF_M   = 3
KUKMIN_MD_TABLE = 'CDS_AMT.EMART_PRODUCT_LIST_U_CSV'

print("================================================")
print("============= Argument setting =================")
print("DNA_YM : ",DNA_YM,
      "\nBF_M : ",BF_M,sep="")
print("================================================")

DATE= f'''TO_CHAR(R.BSN_DT,'YYYYMM') BETWEEN ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'),-{BF_M}) AND ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'), -1)'''

YM_WCNT=DNA_YM+'01'
EVENT_MONTH= DNA_YM

============= Argument setting =================
DNA_YM : 202009
BF_M : 3


# 데이터 추출

In [4]:
query1=f'''
-- SKU Basket 비중
WITH BASE_RCIP_CUST_ID AS ( SELECT A.CUST_ID,A.BSN_DT,A.PRDT_CD,A.AFLCO_CD ,A.BIZTP_CD
                            FROM CDS_DW.TB_DW_RCIPT_DETAIL A 
                            WHERE   A.AFLCO_CD ='001'
                                AND A.BIZTP_CD ='10'
                                AND A.RL_SALE_TRGT_YN = 'Y'
                                AND TO_CHAR(A.BSN_DT,'YYYYMM') BETWEEN ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'),-3) AND ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'), 0) --고객산출 기간변경
                                AND A.CUST_ID IS NOT NULL)
   ,EMART_PRODUCT_LIST AS (  SELECT *
                             FROM  {KUKMIN_MD_TABLE}
                             WHERE YM='{DNA_YM}')
SELECT  A.CUST_ID 																					   	       AS CUST_ID
      , (CASE WHEN SUM(PURCHA_SKU) IS NULL THEN 0 ELSE SUM(PURCHA_SKU) END)/SUM(TOT_SKU_VISIT)         	       AS RATIO_SKU
      , (CASE WHEN MAX(C.AVG_MAIN_PURCHS_SCORE) IS NULL THEN 0 ELSE MAX(C.AVG_MAIN_PURCHS_SCORE) END)          AS DCODE_MAIN_PURCHS_SCORE_F
      , (CASE WHEN MAX(D.TOTAL_EVENT_TYP_PRE_UNITY) IS NULL THEN 0 ELSE MAX(D.TOTAL_EVENT_TYP_PRE_UNITY) END)  AS TOTAL_EVENT_TYP_PRE_UNITY
      , (CASE WHEN MAX(E.LOWPC_PREFER) IS NULL THEN 0 ELSE MAX(E.LOWPC_PREFER) END)     			           AS LOWPC_PREFER
      , CAST(MAX(CASE WHEN F.K_PRDT_CD > 0 THEN 1 ELSE 0 END ) AS DECIMAL)                                     AS PURCHA_MONTH 
-- A: 최근 3개월 동안 구매이력이 있는 고객의 방문횟수
FROM (
        SELECT CUST_ID,PRDT_CD,COUNT(DISTINCT BSN_DT) AS TOT_SKU_VISIT
        FROM BASE_RCIP_CUST_ID
        WHERE TO_CHAR(BSN_DT,'YYYYMM') BETWEEN ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'),-3) AND ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'), -1)
        GROUP BY CUST_ID,PRDT_CD
      ) A 
-- B: 고객별 국민가격 상품 구매횟수
LEFT JOIN (SELECT A.CUST_ID,A.PRDT_CD,COUNT(DISTINCT A.BSN_DT) AS PURCHA_SKU
            FROM BASE_RCIP_CUST_ID A
            JOIN EMART_PRODUCT_LIST B ON A.PRDT_CD = B.PRDT_CD
            WHERE B.PRDT_CD IS NOT NULL 
            AND TO_CHAR(A.BSN_DT,'YYYYMM') BETWEEN ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'),-3) AND ADD_MONTHS (TO_CHAR ('{DNA_YM}', 'YYYYMM'), -1)
            GROUP BY A.CUST_ID,A.PRDT_CD 
          ) B ON A.CUST_ID = B.CUST_ID AND A.PRDT_CD = B.PRDT_CD 
-- C : 주 구매 스코어
LEFT JOIN (
            SELECT CUST_ID,AVG(MAIN_PURCHS_SCORE) AS AVG_MAIN_PURCHS_SCORE
            FROM TB_AMT_CUST_PRDT_DNA_DATA 
            WHERE YM_WCNT='{YM_WCNT}'
            AND AFLCO_CD ='001'
            AND BIZTP_CD ='10' 
            AND PRDT_DCODE_CD IN (SELECT DISTINCT PRDT_DCODE_CD FROM EMART_PRODUCT_LIST)
            GROUP BY CUST_ID
          ) C ON A.CUST_ID = C.CUST_ID
-- D : 행사 이벤트 선호도 
LEFT JOIN ( 
            SELECT CUST_ID,CASE WHEN EVENT_TYP_PRE_UNITY IS NULL THEN 0 ELSE EVENT_TYP_PRE_UNITY END AS TOTAL_EVENT_TYP_PRE_UNITY
            FROM TB_AMT_BIZTP_CUST_DNA_DATA
            WHERE YM_WCNT ='{YM_WCNT}' --월 변경
            AND AFLCO_CD ='001'
            AND BIZTP_CD ='10'
            ORDER BY EVENT_TYP_PRE_UNITY DESC
           ) AS D ON A.CUST_ID = D.CUST_ID
-- E : 저가 선호도(1-고가선호도)
LEFT JOIN (
            SELECT CUST_ID,CASE WHEN (1-HGHPC_PREFER) IS NULL THEN 0 ELSE (1-HGHPC_PREFER) END AS LOWPC_PREFER
            FROM TB_AMT_AFLCO_CUST_DNA_DATA
            WHERE YM_WCNT ='{YM_WCNT}' 
            AND AFLCO_CD ='001'
          ) AS E ON A.CUST_ID = E.CUST_ID
-- ★  F : Target (학습에만 사용되는 Tagging값 운영에는 필요X)
LEFT JOIN (SELECT A.CUST_ID,COUNT(B.PRDT_CD) AS K_PRDT_CD
            FROM BASE_RCIP_CUST_ID A
            LEFT JOIN EMART_PRODUCT_LIST B ON A.PRDT_CD = B.PRDT_CD
            WHERE B.PRDT_CD IS NOT NULL 
            AND TO_CHAR(A.BSN_DT,'YYYYMM') = '{DNA_YM}'
            GROUP BY A.CUST_ID
          ) F ON A.CUST_ID = F.CUST_ID 
GROUP BY A.CUST_ID
    '''
print(query1)


-- SKU Basket 비중
WITH BASE_RCIP_CUST_ID AS ( SELECT A.CUST_ID,A.BSN_DT,A.PRDT_CD,A.AFLCO_CD ,A.BIZTP_CD
                            FROM CDS_DW.TB_DW_RCIPT_DETAIL A 
                            WHERE   A.AFLCO_CD ='001'
                                AND A.BIZTP_CD ='10'
                                AND A.RL_SALE_TRGT_YN = 'Y'
                                AND TO_CHAR(A.BSN_DT,'YYYYMM') BETWEEN ADD_MONTHS (TO_CHAR ('202009', 'YYYYMM'),-3) AND ADD_MONTHS (TO_CHAR ('202009', 'YYYYMM'), 0) --고객산출 기간변경
                                AND A.CUST_ID IS NOT NULL)
   ,EMART_PRODUCT_LIST AS (  SELECT *
                             FROM  CDS_AMT.EMART_PRODUCT_LIST_U_CSV
                             WHERE YM='202009')
SELECT  A.CUST_ID 																					   	       AS CUST_ID
      , (CASE WHEN SUM(PURCHA_SKU) IS NULL THEN 0 ELSE SUM(PURCHA_SKU) END)/SUM(TOT_SKU_VISIT)         	       AS RATIO_SKU
      , (CASE WHEN MAX(C.AVG_MAIN_PURCHS_SCORE) IS NULL THEN 0 ELSE MAX(C.AVG_MAIN_PURCHS_S

In [5]:
KUKMIN_PRICE_FEATURE = query_SQL(query1)

---- 347.98173427581787 seconds ------


In [7]:
feature_name  = ['CUST_ID','RATIO_SKU', 'DCODE_MAIN_PURCHS_SCORE_F', 'TOTAL_EVENT_TYP_PRE_UNITY', 'LOWPC_PREFER','PURCHA_MONTH']
KUKMIN_PRICE_FEATURE.columns = feature_name
KUKMIN_PRICE_FEATURE['PURCHA_MONTH'] = KUKMIN_PRICE_FEATURE['PURCHA_MONTH'].astype(int)
KUKMIN_PRICE_FEATURE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6831894 entries, 0 to 6831893
Data columns (total 6 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   CUST_ID                    object 
 1   RATIO_SKU                  float64
 2   DCODE_MAIN_PURCHS_SCORE_F  float64
 3   TOTAL_EVENT_TYP_PRE_UNITY  float64
 4   LOWPC_PREFER               float64
 5   PURCHA_MONTH               int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 312.7+ MB


## Split train / test set

In [8]:
KUKMIN_PRICE_FEATURE_processed_num =KUKMIN_PRICE_FEATURE.apply(pd.to_numeric, errors = 'coerce').fillna(0)
KUKMIN_x = KUKMIN_PRICE_FEATURE[feature_name[1:-1]]
KUKMIN_y = KUKMIN_PRICE_FEATURE['PURCHA_MONTH']

x_train, x_test, y_train, y_test = train_test_split(KUKMIN_x, KUKMIN_y,test_size=0.2,random_state=123,stratify=KUKMIN_y)

print('x_train length : %s'%len(x_train))
print('y_train length : %s'%len(y_train))
print('x_test  length : %s'%len(x_test))
print('y_test  length : %s'%len(y_test))
print('all   0 class   : %s | all   1 class : %s '%(Counter(KUKMIN_y)[0],Counter(KUKMIN_y)[1]))
print('train 0 class   : %s | train 1 class : %s '%(Counter(y_train)[0] ,Counter(y_train)[1]))
print('test  0 class   : %s | test  1 class : %s '%(Counter(y_test)[0]  ,Counter(y_test)[1]))

x_train length : 5465515
y_train length : 5465515
x_test  length : 1366379
y_test  length : 1366379
all   0 class   : 6482085 | all   1 class : 349809 
train 0 class   : 5185668 | train 1 class : 279847 
test  0 class   : 1296417 | test  1 class : 69962 


# weight(Logistic Regression/RandomForest) 산출

In [103]:
def feature_weight_calculate(df,cat):
    data = df.reindex(columns=feature_name[1:-1])
    df_cols = data.columns
    data['ESTIMATOR'] = cat
    denominator  = (data['RATIO_SKU']+data['DCODE_MAIN_PURCHS_SCORE_F']+data['TOTAL_EVENT_TYP_PRE_UNITY']+data['LOWPC_PREFER'])
    scale_cols = ['ESTIMATOR']
    for i in range(0,len(df_cols)):
        cols = str(df_cols[i])+"_SCALED"
        scale_cols.append(cols)
        data[cols] = data[df_cols[i]]/denominator
    return data[scale_cols]

# Feature importance Weight 

### (Ramdom Forest -> LGBM 으로 변경) 속도 및 성능차이

In [117]:
def FEATURE_IMPORTANCE_KUKMIN(KUKMIN_x,KUKMIN_y,x_test,cat):
    if cat == 'Logit':
        log= LogisticRegression()
        log.fit(KUKMIN_x,KUKMIN_y)
        y_pred=log.predict(x_test)
        y_pred_prob = log.predict_proba(x_test)
        importances_rf=pd.DataFrame(log.coef_)
        importances_rf.columns = KUKMIN_x.columns       
    else:
        clf = LGBMClassifier( n_estimators= 500,
                              num_leaves  = 200, objective=None, random_state=123, reg_alpha=0.0,
                              subsample_for_bin=200000, subsample_freq=0)

        clf.fit(KUKMIN_x, KUKMIN_y)
        result_rfc = permutation_importance(clf, KUKMIN_x,KUKMIN_y,n_repeats=5, n_jobs=20)
        sorted_idx = result_rfc.importances_mean.argsort()
        importances_rf = pd.DataFrame(result_rfc.importances_mean[sorted_idx], index=KUKMIN_x.columns[sorted_idx]).sort_values(0, ascending=False).iloc[:5]    
        pi_cols = importances_rf.reset_index()['index'].values
        pi_weight = importances_rf.loc[:,0].values.tolist()
        importances_rf = pd.DataFrame({str(pi_cols[0]) : float(pi_weight[0]),
                          str(pi_cols[1]) : float(pi_weight[1]),
                          str(pi_cols[2]) : float(pi_weight[2]),
                          str(pi_cols[3]) : float(pi_weight[3]),},index=[0])
    return importances_rf

# Logistic Regression

In [118]:
KUKMIN_PRICE_COEFFICIENT = FEATURE_IMPORTANCE_KUKMIN(KUKMIN_x, KUKMIN_y,x_test,'Logit')
LOGISTIC_weight          = feature_weight_calculate(KUKMIN_PRICE_COEFFICIENT,'Logistics')
LOGISTIC_weight

# Permutation Importance

In [124]:
KUKMIN_PRICE_IMPORTANCE=FEATURE_IMPORTANCE_KUKMIN(KUKMIN_x, KUKMIN_y,x_test,'LGBM')
Boosting_weight=feature_weight_calculate(KUKMIN_PRICE_IMPORTANCE,'LGBM')
Boosting_weight